# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_city_data = pd.read_csv("../WeatherPy/WeatherCityData.csv")

#types_df = pd.read_csv(weather_city_data)

weather_city_data.dropna()
weather_city_data.head(10)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Chui,0,UY,1583037421,88,-33.70,-53.46,291.78,1.97
1,1,Vila Franca do Campo,40,PT,1583037421,88,37.72,-25.43,290.15,6.70
2,2,New Norfolk,75,AU,1583037421,49,-42.78,147.06,295.93,4.60
3,3,Cidreira,0,BR,1583037421,75,-30.18,-50.21,295.87,7.07
4,4,Geraldton,0,AU,1583037422,33,-28.77,114.60,305.15,8.20
5,5,Busselton,47,AU,1583037422,30,-33.65,115.33,299.82,8.61
6,6,Tuktoyaktuk,40,CA,1583037423,74,69.45,-133.04,241.15,3.60
7,7,Kyren,99,RU,1583037424,81,51.68,102.14,261.27,2.33
8,8,Bredasdorp,35,ZA,1583037424,87,-34.53,20.04,291.48,3.60
9,9,West Point,1,US,1583037424,75,33.61,-88.65,283.15,2.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

locations = weather_city_data.head(10)[["Lat","Lng"]]
humidity = weather_city_data.head(10)["Humidity"].astype(float)

In [9]:
#Plot Heatmap
fig = gmaps.figure(center = [0,0], zoom_level = 1)

#Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10,
                                   point_radius=4)
#Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
new_weather_df = pd.DataFrame(weather_city_data, columns = ["City","Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_weather_df["Max Temp"] <= 80) & (new_weather_df["Max Temp"] > 70)
wind_speed = new_weather_df["Wind Speed"] < 10
cloudiness = new_weather_df["Cloudiness"] == 0

new_weather_df.head()

,City,Max Temp,Wind Speed,Cloudiness
0,Chui,291.78,1.97,0
1,Vila Franca do Campo,290.15,6.70,40
2,New Norfolk,295.93,4.60,75
3,Cidreira,295.87,7.07,0
4,Geraldton,305.15,8.20,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = pd.DataFrame(weather_city_data, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Chui,UY,-33.70,-53.46,
1,Vila Franca do Campo,PT,37.72,-25.43,
2,New Norfolk,AU,-42.78,147.06,
3,Cidreira,BR,-30.18,-50.21,
4,Geraldton,AU,-28.77,114.60,


In [12]:
parameters_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
parameters_search = "hotels"
parameters_radius = 5000
parameters_type = "hotel"

parameters_dict = {
    "location": parameters_coordinates,
    "keyword": parameters_search,
    "radius": parameters_radius,
    "type": parameters_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our parameters dictionary
response = requests.get(base_url, params=parameters_dict)

In [13]:
hotel_df.iloc[0][0]

'Chui'

In [ ]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
parameters_dict = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    parameters_dict["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=parameters_dict)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))

    #try:
     #   hotel_name.append(name_address['results'][0]['name'])
    #except IndexError:
     #   hotel_name.append(np.nan)
        
hotel_name.head()

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.6881437,
                    "lng": -53.4465252
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.68670522010728,
                        "lng": -53.44534982010727
                    },
                    "southwest": {
                        "lat": -33.68940487989273,
                        "lng": -53.44804947989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "88c19c5678b2d85eff6b22b56cbb56a2c56bd14f",
            "name": "Bertelli Chu\u00ed Hotel",
            "photos": [
                {
                    "height": 720,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/102937040773061901

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 37.7149918,
                    "lng": -25.4363034
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.71634212989272,
                        "lng": -25.43496007010728
                    },
                    "southwest": {
                        "lat": 37.71364247010728,
                        "lng": -25.43765972989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "ab4016aab9410c882b21f7977ddcef120b218b55",
            "name": "Apartamentos Francisco Martins",
            "photos": [
                {
                    "height": 3492,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/10246157997518

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.78189,
                    "lng": 147.0710573
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.78035257010728,
                        "lng": 147.0722008298928
                    },
                    "southwest": {
                        "lat": -42.78305222989272,
                        "lng": 147.0695011701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "22c264537e26ba6d15af7ec885708442a9e2a154",
            "name": "Derwent Valley Resort",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1152,
                    "html_attributions": [
                 

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -30.1472761,
                    "lng": -50.1917033
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.14594177010727,
                        "lng": -50.19030122010728
                    },
                    "southwest": {
                        "lat": -30.14864142989272,
                        "lng": -50.19300087989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "d9c09d8973e39d582d5c26085cd3ad0ac7c22896",
            "name": "Pousada Vila Di Italy",
            "photos": [
                {
                    "height": 4128,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/10541580773035068133

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAO4ZTENwxKUFV9F2jAlSjFqO53RnnelfhGAW6_0YUcDOYMjSIFBFtqQmJG4TmPz9NPo-ZezuZhRg_GLeXz32e7XXSR7JyQQk_VkIvOCkCDFJse87LZ2PQrGpQlFZw-NcNuRqFlCKTUNKggbWMH-XCMyy9MUuW-rY4elnXx8_21gQM--RM-Ezx981lnNUsn9T2w9yj6S3Sn0wdoErYFKs0UQ9ZaB_egmXylYfCD4ytDaszfkGI6OzGIL5L2BbQNMTJuDYrExCC7VlvMPbibeY79HtMwBhHg-vwSW5srDt8-yyedoU-Lu4e6bjuxVollOaztYptwY4Rq9q37kE_K3_wnw8WKx-G4IwsJSanjzk_07L7wV-L1ZuA9zZ5eCUtJDBkPnPyPozCuElyAmA-FtHi70SEIaH6RUSVIx3WXm5scmrdC4aFEYF_a1mA89-Gxp0srS0BC5out27",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -28.723692,
                    "lng": 114.620747
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.72211487010728,
                        "lng": 114.6221013298927
                    },
                    "southwest": {
                        "lat": -28.72481452989272,
                     

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -24.695365,
                    "lng": 44.74424
                },
                "viewport": {
                    "northeast": {
                        "lat": -24.69380357010728,
                        "lng": 44.74552307989273
                    },
                    "southwest": {
                        "lat": -24.69650322989272,
                        "lng": 44.74282342010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0f65a8b6583a6b35659c2d253944d6975dbb0c39",
            "name": "Angora Hotel",
            "photos": [
                {
                    "height": 3096,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111894283854262680694\">Jean Elys\u

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -13.713642,
                    "lng": -76.218796
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.71227722010728,
                        "lng": -76.21754662010729
                    },
                    "southwest": {
                        "lat": -13.71497687989272,
                        "lng": -76.22024627989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "8c16696e37bdf558babf752e02e071b5b96c25c9",
            "name": "PiscoMar Peru",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2048,
                    "html_attributions": [
                       

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -46.600178,
                    "lng": 168.3461216
                },
                "viewport": {
                    "northeast": {
                        "lat": -46.59876597010727,
                        "lng": 168.3475455798927
                    },
                    "southwest": {
                        "lat": -46.60146562989272,
                        "lng": 168.3448459201072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0873e7114cd46c000340e6ff0b771453c8539d2b",
            "name": "Foveaux Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1536,
                    "html_attributions": [
                        

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 10.422007,
                    "lng": -64.213876
                },
                "viewport": {
                    "northeast": {
                        "lat": 10.42318867989272,
                        "lng": -64.21235197010728
                    },
                    "southwest": {
                        "lat": 10.42048902010728,
                        "lng": -64.21505162989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f3d1c4c219de630b35cae86d749e4141de9cc580",
            "name": "Hotel Paradise Cuman\u00e1",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3000,
                    "html_attributions": [
             

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAPh6nIt3kV3TkFv2DxUKiTFYehIMVBJyOSZgueRO1etgagaFcxjfC3lCLMg2mWzTQUIak_V31vV-TctxwMCQMmhot46oQfUvuYoTLW0ZIVZ33wXeu4lgw4ZSZeAT_LGZkfCn991QNL5-ETY6D2X5JRH6hUQSO7cwBijsFIj5mU76txPcFfqUOV_lqrGU0q9y3X2y-CsjqbUzKePnqr1YqQXu4rtCvSg11hD-jskt3u8AEoRjcbD2A9jU1dC02jLuRKTPtM43Svr_9-3dYUnGi0gP2jBBbvTGePOHYxT_c6dyj6v7d6ReKA5qjIyXI_I52Ibw6IOXPKqOSL0u-R2VSlLU3siY3SHJHUyTXyTCF-Ys3w764a_SZLQOjRe6THGu5PtZJu-UjvwGFc-orH-TNGESEFi8fUTlqRBO7Qm9j5VSfYYaFNSFJ8zdc-sZpLDbdfmqbeEpAAJy",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -54.84185799999999,
                    "lng": -68.37251499999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -54.84041852010728,
                        "lng": -68.37121187010727
                    },
                    "southwest": {
                        "lat": -54.84311817989272,
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

# Display Map